In [1]:
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfile
import tkinter as tk
from tkinter import ttk
import cv2 as cv
import numpy as np
import os
import math
from skimage import io, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.restoration import denoise_tv_chambolle
from sklearn.mixture import GaussianMixture as GMM
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TKAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from skimage.filters import threshold_multiotsu
from scipy import ndimage as nd
from PIL import ImageTk,Image 
from scipy import ndimage
from skimage import measure, color, io
import cv2
from matplotlib import figure
from skimage import measure, io, img_as_ubyte
from skimage.color import label2rgb, rgb2gray
import pandas as pd



class segmentation(Tk):
    
    def __init__(self):
        super().__init__()

        # Set the geometry of tkinter frame
        (self.geometry("700x350"))

        ttk.Button(self, text="Select_Images for segmentation", command=self.main_image).place(x = 10, y = 10)
        # select the image that needs to be segment (multiple or single image can be select)
        ttk.Button(self, text="Reference Image", command=self.reference_image).place(x = 215, y= 10)
        # select the unskewed image that will unskewed the original image (only single image can be select)
        
        ttk.Button(self, text="Region of interest", command=self.region_image).place(x = 340, y= 10)
        # select the region of interest image that will detect the same region in the original (only single image can be select)
        
        self.filter_button() # filters option
        self.K_value() # to segment the image in k number of clusters 
        self.segmentation_button() # options for different segment algorithm 
        self.watershed_values() # algorithm for closing the open voids 


    def main_image(self):
       self.image = filedialog.askopenfilenames(initialdir="/", title="Select A File", filetypes=(("jpg files", "*.jpg"),("all files", "*.*"))) 
        # button to select the image for segmentation
    def reference_image(self):
       self.ref_image = filedialog.askopenfilename(initialdir="/", title="Select A File", filetypes=(("jpg files", "*.jpg"),("all files", "*.*")))
       # button to select the unskewed image 
       self.rotation() # algorithm for unskewed the original image  
    def region_image(self):
      self.filename = filedialog.askopenfilename(initialdir="/", title="Select A File", filetypes=(("jpg files", "*.jpg"),("all files", "*.*")))
      # button to select the region of interest image 
        
      self.Region_of_interest() # algorithm to detect the region of interest in the original 

    def rotation (self):     

           self.unskewed_image=[]      
           for i in self.image: # iterate over the original images

                im1 = cv.imread(i,0)  # skewed image       
                im2 = cv.imread(self.ref_image,0)# reference image

                img1=im1.copy()
                img2=im2.copy()

                
                MIN_MATCHES = 2 # minimum number of matches should be between the skewed image and reference image 
                
                # Initiate ORB detector
                orb = cv.ORB_create(nfeatures=500)#Registration works with at least 500 points
                
                # find the keypoints and descriptors with orb
                kp1, des1 = orb.detectAndCompute(img1, None) #kp1 --> list of keypoints
                kp2, des2 = orb.detectAndCompute(img2, None) #kp2 --> list of keypoints
                
                index_params = dict(algorithm=6,
                                    table_number=6,
                                    key_size=12,             # algorithm parameters
                                    multi_probe_level=2)
                search_params = {}
                flann = cv2.FlannBasedMatcher(index_params, search_params)
                matches = flann.knnMatch(des1, des2, k=3)

                # As per Lowe's ratio test to filter good matches
                good_matches = []
                for m, n, k in matches:
                    if m.distance < 0.75 * n.distance:
                        good_matches.append(m)

                if len(good_matches) > MIN_MATCHES:
                    src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    m, mask = cv.findHomography(src_points, dst_points, cv.RANSAC, 5.0)
                    corrected_img = cv2.warpPerspective(img1, m, (img2.shape[1], img2.shape[0]),flags=cv2.INTER_LINEAR)
                angle = - math.atan2(m[0,1], m[0,0]) * 180 / math.pi
                if angle < -45:
                        angle = -(90 + angle)
                        # otherwise, just take the inverse of the angle to make it positive
                        
                else:
                    angle = -angle
                (h, w) = im2.shape[:2]
                center = (w // 2, h // 2)
                M = cv.getRotationMatrix2D(center, angle, 1.0)
                rotated = cv.warpAffine(img1, M, (w, h),
                    flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)
                
                self.unskewed_image.append(rotated)
                
                if ((i==self.image[0]).all()):
                        cv2.imshow('original_image', im1)
                        cv2.imshow('Corrected image', rotated)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()


    def Region_of_interest(self):


            def template_matching(img2,template):
                w, h = template.shape[::-1]
                # All the 6 methods for comparison in a list
                #methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR','cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
                methods = [ 'cv.TM_SQDIFF_NORMED']
                for meth in methods:
                    self.img = img2.copy()
                    method = eval(meth)
                    # Apply template Matching
                    res = cv2.matchTemplate(self.img,template,method)
                    # For TM_SQDIFF, Good match yields minimum value; bad match yields large values
                    # For all others it is exactly opposite, max value = good fit.
                    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
                    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
                    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                                top_left = min_loc
                    else:
                        top_left = max_loc #Change to max_loc for all except for TM_SQDIFF
                    bottom_right = (top_left[0] + w, top_left[1] + h)
                    # Prepare crop area
                    width, height = bottom_right[0]-top_left[0], bottom_right[1]-top_left[1]
                    x, y = top_left[0], top_left[1]
                    # Crop image to specified area using slicing
                    self.crop_img = self.img[y:y+height, x:x+width]
                    # Show image
                    
                    cv.imshow("cropped", self.crop_img)               
                    cv.waitKey(0)
                    cv.destroyAllWindows()
                    return self.crop_img
                    
                    
#                     try:
#                         if ((img2==self.image[0]).all()):
#                             cv.imshow("cropped", self.crop_img)               
#                             cv.waitKey(0)
#                             cv.destroyAllWindows()
#                         return self.crop_img
                    
#                     except AttributeError: 
#                             if ((img2==self.unskewed_image[0]).all()):
#                                 cv.imshow("cropped", self.crop_img)
#                                 cv.waitKey(0)
#                                 cv.destroyAllWindows()
#                             return self.crop_img

            try:
                self.region=[]
                for i in self.image:
                    img2 =  cv.imread(i,0)
                    template = cv.imread(self.filename,0)
                    matching_region= template_matching(img2,template)
                    self.region.append(matching_region)

            except AttributeError:
                self.region=[]
                for i in self.unskewed_image:
                    img2 = cv.imread(i,0) 
                    template = cv.imread(self.filename,0)
                    matching_region=template_matching(img2,template)
                    self.region.append(matching_region)


    def filter_button(self):

            def callback(event):

                  if self.comboExample.get() =='circle_detection': 
                                self.circle_pop()

                  if self.comboExample.get() == 'Blurr_filter':
                        self.gaussian()

                  if self.comboExample.get() == 'sharp':
                        self.sharp()


            self.labelTop = tk.Label(self,
                            text = "Choose the filter ")

            self.labelTop.place(x=470,y=10)
            self.comboExample = ttk.Combobox(self, 
                                    values=[
                                            "circle_detection", 
                                            "Blurr_filter",
                                            "sharp",
                                            ])
            self.comboExample.place(x=470,y=30)

            self.comboExample.bind("<<ComboboxSelected>>", callback)

    def calculation(self):

            self.circle=[]
            for i in self.region:
                self.image=i.copy()
                circles = cv.HoughCircles(self.image, cv.HOUGH_GRADIENT,float(self.a.get()),int(self.b.get()),int(self.c.get()),int(self.d.get()))
                detected_circles = np.uint16(np.around(circles))
                for (x, y, r) in detected_circles[0,:]:
                    cv.circle(self.image, (x,y), int(r)+int(self.g.get()),255,-1)
                self.circle.append(self.image)
                
                #if ((self.image==self.region[0]).all()):
                cv.imshow('output',self.image)
                cv.waitKey(0)
                cv.destroyAllWindows()
            self.top.destroy()


    def circle_pop(self):

    #Create a Toplevel window
        self.top= Toplevel(self)
        self.top.geometry("400x400")

        Label(self.top, text="Enter inverse ratio of accumulator resolution and image resolution", font=('Calibri 10')).pack()
        self.a=Entry(self.top, width=35)
        self.a.insert(0, float(1.67))
        self.a.pack()

        Label(self.top, text="Enter Minimum_Distance", font=('Calibri 10')).pack()
        self.b=Entry(self.top, width=35)
        self.b.insert(0, int(50))
        self.b.pack()

        Label(self.top, text="Enter  Parameter_1", font=('Calibri 10')).pack()
        self.c=Entry(self.top, width=35)
        self.c.insert(0, int(100))
        self.c.pack()

        Label(self.top, text="Enter  Parameter_2", font=('Calibri 10')).pack()
        self.d=Entry(self.top, width=35)
        self.d.insert(0, int(50))
        self.d.pack()


        Label(self.top, text="Enter  additional radius for circle", font=('Calibri 10')).pack()
        self.g=Entry(self.top, width=35)
        self.g.insert(0, int(22))
        self.g.pack()



        self.button= Button(self.top, text="Ok", command=self.calculation)
        self.button.pack(pady=5, side= TOP)

    def cal (self):
        self.blurr=[]
        for i in self.region:
            img=i.copy()
            self.denoise_img = denoise_tv_chambolle(img, weight=float(self.a.get()), eps=float(self.b.get()), n_iter_max=int(self.c.get()), multichannel=False)
            self.blurr.append(self.denoise_img)
            """
            denoise_tv_chambolle(image, weight=0.1, eps=0.0002, n_iter_max=200, multichannel=False)
            weight: The greater weight, the more denoising (at the expense of fidelity to input).
            eps: Relative difference of the value of the cost function that determines the stop criterion. 
            n_iter_max: Max number of iterations used for optimization
            """

            cv.imshow('output',self.denoise_img)
            cv.waitKey(0)
            cv.destroyAllWindows()
        self.pop.destroy()

    def gaussian(self):

        self.pop= Toplevel(self)
        self.pop.geometry("400x400")

        Label(self.pop, text="Enter weight", font=('Calibri 10')).pack()
        self.a=Entry(self.pop, width=35)
        self.a.insert(0, float(0.015))
        self.a.pack()

        Label(self.pop, text="Enter number of eps", font=('Calibri 10')).pack()
        self.b=Entry(self.pop, width=35)
        self.b.insert(0, float(0.0005))
        self.b.pack()

        Label(self.pop, text="Enter number of iteration", font=('Calibri 10')).pack()
        self.c=Entry(self.pop, width=35)
        self.c.insert(0, int(200))
        self.c.pack()


        self.button= Button(self.pop, text="Ok", command=self.cal)
        self.button.pack(pady=5, side= TOP)


    def calu(self):
        self.sharp=[]
        for i in self.region:
            img=i.copy()
            self.unsharped_img = unsharp_mask(img, radius=int(self.a.get()), amount=int(self.b.get()))
            self.sharp.append(self.unsharped_img)
            cv.imshow('output',self.unsharped_img)
            cv.waitKey(0)
            cv.destroyAllWindows()
            self.pop.destroy()    

    def sharp(self):

        self.pop= Toplevel(self)
        self.pop.geometry("400x400")

        Label(self.pop, text="Enter radius", font=('Calibri 10')).pack()
        self.a=Entry(self.pop, width=35)
        self.a.insert(0, int(3))
        self.a.pack()

        Label(self.pop, text="Enter the amount", font=('Calibri 10')).pack()
        self.b=Entry(self.pop, width=35)
        self.b.insert(0, int(1))
        self.b.pack()

        self.button= Button(self.pop, text="Ok", command=self.calu)
        self.button.pack(pady=5, side= TOP)



    def OTSU_calculation(self,image):
            self.n=int(self.z.get()) # number of clusters
            thresholds = threshold_multiotsu(image, classes=self.n)
            
            # Digitize (segment) original image into multiple classes.
            #np.digitize assign values 0, 1, 2, 3, ... to pixels in each class.
            regions = np.digitize(image, bins=thresholds)
            dic={}
            for i in range(self.n):
                a='seg'+str(i+1)
                b=(regions == int(i))  # to create the different region number for every cluster
                dic[a]=b
            
            
            #We can use binary opening and closing operations to clean up. 
            #Open takes care of isolated pixels within the window
            #Closing takes care of isolated holes within the defined window
            di_open={}
            di_close={}
            for j in range(self.n):
                c='seg'+str(j+1)+'_'+'opened'
                e='seg'+str(j+1)
                d=nd.binary_opening(dic[e], np.ones((3,3)))
                di_open[c]=d
                g='seg'+str(j+1)+'_'+'closed'
                f=nd.binary_closing(di_open[c], np.ones((3,3)))
                di_close[g]=f

            all_segments_cleaned = np.zeros((image.shape[0], image.shape[1])) 

            
            # here, we assign the each cluster white color with a region name and if we show lets say region 1 then in the
            # image you will the cluster or region one with white color and the remainings are in black. This helps to visualize
            # the clusters separately. 
            self.dic_show={}
            i=0
            for x, y in di_close.items():
                all_segment=all_segments_cleaned.copy()
                all_segment[y] = 255
                ig= 'region'+str(i)
                self.dic_show[ig]= all_segment
                i+=1
                
                   
            # when multiple images are there, in the GUI it will pop the first image only to the user
            try:
                if ((image==self.circle[0]).all()):             
                    for x,y in self.dic_show.items():
                        cv2.imshow(x, y)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                
            except AttributeError:
                if ((image==self.blurr[0]).all()):
                    for x,y in self.dic_show.items():
                        cv2.imshow(x, y)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                
            except AttributeError:
                if ((image==self.sharp[0]).all()):
                    for x,y in self.dic_show.items():
                        cv2.imshow(x, y)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                     
            self.region= all_segments_cleaned.reshape(image.shape) #All the noise should be cleaned now





    def OTSU(self):
        try:
            self.final_dict={}
            c=0
            for i in self.circle:
                    image = i.copy()
                    self.OTSU_calculation(image)
                    self.final_dict[c]=self.dic_show
                    c+=1
                    
            self.pop.destroy()
                    

        except AttributeError:
            self.final_dict={}
            c=0
            for i in self.blurr:
                image=i.copy()
                self.OTSU_calculation(image)
                self.final_dict[c]=self.dic_show
                c+=1
                
            self.pop.destroy()

        except AttributeError:
            self.final_dict={}
            c=0
            for i in self.sharp:
                image=i.copy()
                self.OTSU_calculation(image)
                self.final_dict[c]=self.dic_show
                c+=1
                
            self.POP.destroy()


    def K_value(self):

            def K_value_graph():
                img=self.region[0]
                img2 = img.reshape((-1,1))

                #gmm_model = GMM(2, covariance_type='tied').fit(img2)
                #The above line generates GMM model for n=4
                #Now let us call the bic method (or aic if you want).

                #bic_value = gmm_model.bic(img2)  #Remember to call the same model name from above)
                #You should see bic for GMM model generated using n=2.
                #Do this exercise for different n values and plot them to find the minimum.


                #Now, to explain m.bic, here are the lines I used in the video. 
                n_components = np.arange(3,8)
                gmm_models = [GMM(n, covariance_type='tied').fit(img2) for n in n_components]
                plt.plot(n_components, [m.bic(img2) for m in gmm_models], label='BIC')
                f=Figure(figsize=(5,5),dpi=20)
                canvas=FigureCanvasTkAgg(f,self)
                #canvas.show()
                #canvas.get_tk_widget().pack(side=tk.TOP,fill=tk.BOTH,expand=True)


            def write_slogan():

                self.pop= Toplevel(self)
                self.pop.geometry("600x300")

                Label(self.pop, text="Enter K value", font=('Calibri 10')).pack()
                self.z=Entry(self.pop, width=30)
                self.z.pack()

                self.button= Button(self.pop, text="Show segemented image", command=self.OTSU)
                self.button.pack(pady=5, side= TOP)

                self.button= Button(self.pop, text="Suggest K value", command=K_value_graph)
                self.button.pack(pady=5, side= TOP)


            slogan = tk.Button(self,text="K_value",command=write_slogan).place(x=10,y=70)



    def segmentation_button(self):

            def back(e):

                  if self.comboExample.get() =='Multi OTSU': 
                                pass
                  if self.comboExample.get() == 'K-means':
                        pass
                  if self.comboExample.get() == 'GMM':
                        pass

            self.labelpop = tk.Label(self,
                            text = "Choose the segmentation method ")

            self.labelpop.place(x=80,y=65)
            self.comboexample = ttk.Combobox(self, 
                                    values=[
                                            "Multi OTSU", 
                                            "K-means",
                                            "GMM",
                                            ])
            self.comboexample.place(x=85,y=90)

            self.comboexample.bind("<<ComboboxSelected>>", back)


    def region_calculation(self):

            def close():

                self.POP.destroy()

            def select_regions():

                self.POP= Toplevel(self)
                self.POP.geometry("300x300")

                Label(self.POP, text="Enter solderbale region", font=('Calibri 10')).pack()
                self.k=Entry(self.POP, width=20)
                self.k.insert(0, int(0))
                self.k.pack()

                Label(self.POP, text="Enter void area", font=('Calibri 10')).pack()
                self.l=Entry(self.POP, width=20)
                self.l.insert(0, int(1))
                self.l.pack()

                self.button= Button(self.POP, text="Ok", command=close)
                self.button.pack(pady=5, side= TOP)

            slogan = tk.Button(self,text="Select regions for calculation",command=select_regions).place(x=280,y=80)

    
    def voids_close(self,void_img,number):
        #img1=img1.astype(np.float32)
        
        img1= cv.cvtColor(self.circle[number],cv2.COLOR_GRAY2RGB)

#         except AttributeError:
            
#             img1= cv.cvtColor(self.blurr[0],cv2.COLOR_GRAY2RGB)
            
#         except AttributeError:
                
#             img1= cv.cvtColor(self.sharp[0],cv2.COLOR_GRAY2RGB)
            

        #Threshold image to binary using OTSU. ALl thresholded pixels will be set to 255
        #ret1, thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        all_segments_cleaned= void_img.copy()
        thresh=   all_segments_cleaned.copy()

        # Morphological operations to remove small noise - opening
        #To remove holes we can use closing
        kernel = np.ones((3,3),np.uint8)
        opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 5)

        from skimage.segmentation import clear_border
        opening = clear_border(opening) #Remove edge touching grains
        #Check the total regions found before and after applying this. 


        #Now we know that the regions at the center of cells is for sure cells
        #The region far away is background.
        #We need to extract sure regions. For that we can use erode. 
        #But we have cells touching, so erode alone will not work. 
        #To separate touching objects, the best approach would be distance transform and then thresholding.

        # let us start by identifying sure background area
        # dilating pixes a few times increases cell boundary to background. 
        # This way whatever is remaining for sure will be background. 
        #The area in between sure background and foreground is our ambiguous area. 
        #Watershed should find this area for us. 
        sure_bg = cv.dilate(opening,kernel,iterations=5)


        # Finding sure foreground area using distance transform and thresholding
        #intensities of the points inside the foreground regions are changed to 
        #distance their respective distances from the closest 0 value (boundary).
        #https://www.tutorialspoint.com/opencv/opencv_distance_transformation.htm
        dist_transform = cv.distanceTransform(opening,cv.DIST_L2,3)

        #Let us threshold the dist transform by 20% its max value.
        #print(dist_transform.max()) gives about 21.9
        ret2, sure_fg = cv.threshold(dist_transform,0.2*dist_transform.max(),255,0)

        #0.2* max value seems to separate the cells well.
        #High value like 0.5 will not recognize some grain boundaries.

        # Unknown ambiguous region is nothing but bkground - foreground
        sure_fg = np.uint8(sure_fg)

        unknown = cv.subtract(sure_bg,sure_fg)

        #Now we create a marker and label the regions inside. 
        # For sure regions, both foreground and background will be labeled with positive numbers.
        # Unknown regions will be labeled 0. 
        #For markers let us use ConnectedComponents. 
        #ret3, markers = cv2.connectedComponents(sure_fg)
        connectivity=8
        n_labels, markers, stats, centroids = cv.connectedComponentsWithStats(sure_fg,
                                                                              connectivity,cv.CV_32S)

         # Create false color image
        colors = np.random.randint(0, 255, size=(n_labels , 3), dtype=np.uint8)
        colors[0] = [0, 0, 0]  # for cosmetic reason we want the background black
        false_colors = colors[markers]
        #One problem rightnow is that the entire background pixels is given value 0.
        #This means watershed considers this region as unknown.
        #So let us add 10 to all labels so that sure background is not 0, but 10
        markers = markers+10

        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        #plt.imshow(markers, cmap='jet')   #Look at the 3 distinct regions.
        markers = markers.astype('int32')
        #markers = markers.astype(np.uint8)
        #Now we are ready for watershed filling. 
        markers = cv.watershed(img1,markers)
        #The boundary region will be marked -1
        #https://docs.opencv.org/3.3.1/d7/d1b/group__imgproc__misc.html#ga3267243e4d3f95165d55a618c65ac6e1


        #Let us color boundaries in yellow. OpenCv assigns boundaries to -1 after watershed.
        img1[markers == -1] = [255,255,255]  

        img2 = color.label2rgb(markers, bg_label=0)
        markers = markers.astype(np.uint8)
#         cv2.imshow('marker', markers)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

        return markers,img2
        
        
    
    
    def watershed(self):

        img=(list((self.final_dict).values()))[0] # it gets the first image in order to help the user to fix the watershed algorithm parameters
        void_img=(list((img).values()))[(int(self.l.get()))] # get the region having the voids
        void_img=void_img.astype(np.uint8)
        loc=0
        markers,img2=self.voids_close(void_img,loc) # apply watershed algorithm on the region having voids
        cv2.imshow('Colored Grains', img2)
        cv2.imshow('marker', markers)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        
        

    def watershed_values(self):

            def closing():
                
                all_img=(list((self.final_dict).values())) # get the all regions of every segmented images 
                
                for j in range (len(all_img)): # iterate over all the images
                    
                    no=j
                    each_img = all_img[j].copy() # here j defines the image number
                    
                    void_img=(list((each_img).values()))[(int(self.l.get()))] # get the region having void of the j number image
                    
                    void_img=void_img.astype(np.uint8)
                    
                    reference_area_img=(list((each_img).values()))[(int(self.k.get()))] # get the region having solderable of the j number image
                    
                    reference_area_img=reference_area_img.astype(np.uint8)
                    
                    markers_void,img2_void=self.voids_close(void_img,no) # apply watershe3d algorithm on void region
                    
                    markers_reference,img2_reference=self.voids_close(reference_area_img,no) # apply watershe3d algorithm on solderable region
                    
                    props_voids_watershed_void = measure.regionprops_table( markers_void,  # calculate the properties of surfaces in the void region
                          properties=['label',
                                      'area',
                                      'eccentricity','centroid','perimeter'])

                    pd.set_option('display.max_rows', None)
                    df_markers_void = pd.DataFrame(props_voids_watershed_void)
                    
                    props_voids_watershed_reference = measure.regionprops_table(markers_reference,  # calculate the properties of surfaces in the solderable region
                          properties=['label',
                                      'area',
                                      'centroid'])

                    pd.set_option('display.max_rows', None)
                    df_markers_reference = pd.DataFrame(props_voids_watershed_reference)
                    
                    
                    
        
                    
                    min_ecen = (float(self.x.get())) # minimum circularity defined for calculating the area of the voids
                    void_area=0
                    false_colors_draw_watershed_void =img2_void.copy()
                    for i ,(x1,y1,eccen,ar,per) in enumerate (zip(df_markers_void['centroid-1'], df_markers_void['centroid-0'],df_markers_void['eccentricity'],df_markers_void['area'],df_markers_void['perimeter'])):
                            p=((12.56)*ar)/(per**2) # calculating the circulating 
                            if p> min_ecen:
                                void_area=void_area+ar
                                formatted_string = "{:.2f}".format(ar)
                                cv2.drawMarker(false_colors_draw_watershed_void, (int(x1), int(y1)),
                                color=(255, 255, 255), markerType=cv2.MARKER_CROSS)
                                cv2.putText(false_colors_draw_watershed_void, str(formatted_string), (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)

                                
                    false_colors_draw_watershed_reference =img2_reference.copy()
                    ref_ar=0
                    for j ,(x2,y2,are) in enumerate (zip(df_markers_reference['centroid-1'], df_markers_reference['centroid-0'],df_markers_reference['area'])):
                        
                            #if p> min_ecen:
                                ref_ar=ref_ar+are
                                #void_area=void_area+ar
                                formatted = "{:.2f}".format(are)
                                cv2.drawMarker(false_colors_draw_watershed_reference, (int(x2), int(y2)),
                                color=(255, 255, 255), markerType=cv2.MARKER_CROSS)
                                cv2.putText(false_colors_draw_watershed_reference, str(formatted), (int(x2), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
                                
                    
                    #print(ref_ar)
                    cv2.imshow('marker', false_colors_draw_watershed_reference)
                    
                    #cv2.destroyAllWindows()
                    
                    cv2.imshow('void', false_colors_draw_watershed_void)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    
        
                    f = Figure(figsize=(20, 20))
                    a = f.add_subplot(111)
                    a.imshow(false_colors_draw_watershed_void)
                    
                    canvas = FigureCanvasTkAgg(f, master=self.TOP)
                    canvas.get_tk_widget().pack(side="top", fill="both", expand=1)
                    canvas._tkcanvas.pack(side="top", fill="both", expand=1)
#                     f,(ax1)= plt.subplots(1,1,figsize=(20,20))
                    
#                     ax1.set_title('false_colors watershed')
#                     ax1.imshow(false_colors_draw_watershed)
                
                    solderable_area= (df_markers_reference['area'].sum())
                    
                    print(solderable_area)
                    print(void_area)
                    void_percentage=((void_area)/ (solderable_area+void_area))
                    print(void_percentage)
                    
        
                #self.TOP.destroy()
                

            def values():

                self.TOP= Toplevel(self)
                self.TOP.geometry("600x600")

                Label(self.TOP, text="Enter Number of Iteration", font=('Calibri 10')).pack()
                self.o=Entry(self.TOP, width=20)
                self.o.insert(0, int(5))
                self.o.pack()

                Label(self.TOP, text="Enter distance", font=('Calibri 10')).pack()
                self.p=Entry(self.TOP, width=20)
                self.p.insert(0, float(0.20))
                self.p.pack()
                
                Label(self.TOP, text="Enter solderbale region", font=('Calibri 10')).pack()
                self.k=Entry(self.TOP, width=20)
                self.k.insert(0, int(0))
                self.k.pack()

                Label(self.TOP, text="Enter void area", font=('Calibri 10')).pack()
                self.l=Entry(self.TOP, width=20)
                self.l.insert(0, int(1))
                self.l.pack()
                
                Label(self.TOP, text="Enter K value", font=('Calibri 10')).pack()
                self.z=Entry(self.TOP, width=20)
                self.z.pack()
                
                Label(self.TOP, text="Enter circularity value", font=('Calibri 10')).pack()
                self.x=Entry(self.TOP, width=20)
                self.x.pack()

                self.button= Button(self.TOP, text="Show image", command=self.watershed)
                self.button.pack(pady=5, side= TOP)


                self.button= Button(self.TOP, text="Ok", command=closing)
                self.button.pack(pady=5, side= TOP)

            slogan = tk.Button(self,text="Values for void closing (watershed) ",command=values).place(x=280,y=80)



  

In [2]:
abc= segmentation()
abc.mainloop()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


373305
53238
0.12481273869223032


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


373305
47885
0.11368978370806525


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


373305
41774
0.1006410827818319
